In [2]:
import os
import pandas as pd
import dask.bag as db
from dask.distributed import Client
import shutil

# Define the directories and output path
directory = r'D:\SamsungSTF\Processed_Data\TripByTrip'
output_path = r'C:\Users\BSL\Desktop\result_files_nan.csv'
destination_directory = r'D:\SamsungSTF\Processed_Data\MissingData_NAN'

# Function to check if a file contains NaN values in specified columns
def check_file(filepath):
    try:
        df = pd.read_csv(filepath)
        if any(col in df.columns for col in ['speed', 'acceleration', 'Power_IV', 'Power']):
            if df[['speed', 'acceleration', 'Power_IV', 'Power']].isnull().any().any():
                return os.path.basename(filepath)
    except Exception as e:
        print(f"Error processing file {filepath}: {e}")
    return None

def main():
    # Start a Dask client with a specified port
    client = Client(dashboard_address=':7869')  # You can choose any available port here

    # Get list of all CSV files in the directory
    csv_files = [os.path.join(directory, filename) for filename in os.listdir(directory) if filename.endswith(".csv")]

    # Create a Dask bag of the file paths
    file_bag = db.from_sequence(csv_files, npartitions=32)
    
    # Map the check_file function to the file paths
    results = file_bag.map(check_file).compute()

    # Filter out None results
    files_with_nan_values = [result for result in results if result]
    files_with_nan_values.sort()
    
    # Save results to a CSV file
    result_df = pd.DataFrame(files_with_nan_values, columns=['filename'])
    result_df.to_csv(output_path, index=False)

    # Move files with NaN values to the destination directory
    for filename in files_with_nan_values:
        source_path = os.path.join(directory, filename)
        if not os.path.exists(destination_directory):
            os.makedirs(destination_directory)
        destination_path = os.path.join(destination_directory, filename)
        shutil.move(source_path, destination_path)
    
    client.close()
    return files_with_nan_values

if __name__ == "__main__":
    result_files = main()
    print(result_files)


C:\Users\BSL\AppData\Roaming\Python\Python312\site-packages\distributed\node.py:182: UserWarning: Port 7869 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 9008 instead
  warnings.warn(


['bms_01241124056-2024-04-trip-2.csv', 'bms_01241228122-2023-02-trip-17.csv', 'bms_01241228122-2023-02-trip-24.csv', 'bms_01241228122-2023-02-trip-37.csv', 'bms_01241228122-2023-03-trip-53.csv', 'bms_01241228122-2023-05-trip-50.csv', 'bms_01241228122-2023-06-trip-25.csv', 'bms_01241228122-2023-09-trip-11.csv', 'bms_01241228122-2023-12-trip-27.csv', 'bms_01241228151-2023-06-trip-30.csv', 'bms_01241228154-2023-02-trip-15.csv', 'bms_01241228154-2023-03-trip-31.csv', 'bms_01241228154-2023-07-trip-7.csv', 'bms_01241228154-2023-08-trip-26.csv', 'bms_01241228154-2023-08-trip-3.csv', 'bms_01241228154-2023-08-trip-41.csv', 'bms_01241228154-2023-09-trip-10.csv', 'bms_01241228154-2023-09-trip-21.csv', 'bms_01241228156-2023-02-trip-12.csv', 'bms_01241228156-2023-02-trip-27.csv', 'bms_01241228156-2023-02-trip-30.csv', 'bms_01241228156-2023-02-trip-9.csv', 'bms_01241228156-2023-05-trip-27.csv', 'bms_01241228156-2023-06-trip-24.csv', 'bms_01241228156-2023-08-trip-43.csv', 'bms_01241228156-2023-09-tri